#                      PYCITY SCHOOLS SNAPSHOT

In [42]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


# District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

### Total number of Schools, Students and Budget

In [2]:
# Get total schools
total_schools = len(school_data_complete["school_name"].unique())

# Get total students
total_students = school_data_complete["Student ID"].count()

# Get total budget
total_budget = school_data["budget"].sum()

### Calculating Scores

In [3]:
# Calculate the average math score
total_math_score = school_data_complete["math_score"].sum()
avg_math_score = total_math_score/total_students

# Calculate the average reading score
total_read_score = school_data_complete["reading_score"].sum()
avg_read_score = total_read_score/total_students

# Calculate the overall passing rate (overall average score), 
#i.e. (avg. math score + avg. reading score)/2
overal_passing_rate = (avg_math_score + avg_read_score)/2

### Calculating Passing (Score 70+) Percentages 

In [4]:
# Find out how many students pass & failed
math_scores = (school_data_complete["math_score"] >= 70).value_counts()

# Make df of scores, reset index of true and false, rename columns
math_scores_df = pd.DataFrame(math_scores).reset_index()
math_scores_rename = math_scores_df.rename(columns={"math_score":"student_count",
                                                   "index":"Passing Status: Math"})


# Filter for students who passed math and get the percentage
passed_math = math_scores_rename.iloc[0,:].sum()
percent_passed_math = passed_math*100/total_students

In [5]:
# Find out how many students pass & failed
reading_scores = (school_data_complete["reading_score"] >= 70).value_counts()
reading_scores

# Make df of scores, reset index of true and false, rename columns
read_scores_df = pd.DataFrame(reading_scores).reset_index()
read_scores_rename = read_scores_df.rename(columns={"reading_score":"student_count",
                                                   "index":"Passing Status: Reading"})

# Filter for students who passed reading and get the percentage
passed_read = read_scores_rename.iloc[0,:].sum()
percent_passed_read = passed_read*100/total_students

### Disctrict Summary DataFrame

In [6]:
# Create a dataframe to hold the above results
district_summary_df = pd.DataFrame({"Total Schools": [total_schools],"Total Students": total_students,
                                    "Total Budget": [total_budget], "Average Math Score": avg_math_score,
                                   "Average Reading Score":avg_read_score, "Passing Math (%)": percent_passed_math,
                                   "Passing Reading(%)":percent_passed_read, 
                                   "Overall Passing Rate(%)":overal_passing_rate})
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Passing Math (%),Passing Reading(%),Overall Passing Rate(%)
0,15,39170,24649428,78.985371,81.87784,74.983406,85.808016,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

### School name and type

In [7]:
# Group by school name and find average
school_group = school_data_complete.groupby("school_name")
score_avg = school_group.mean()

# make size and budget as integers
size_bud_format = score_avg[["size", "budget"]].astype("int64")

# Merge df
merging_df = pd.merge(size_bud_format,score_avg, how="inner", on="school_name")
school_avg = merging_df[["size_x", "budget_x", "reading_score", "math_score"]]
schl_avg_rename = school_avg.rename(columns={"size_x": "Student Population", "budget_x":"Budget",
                                             "reading_score": "Average Reading Score",
                                            "math_score":"Average Math Score"})

# Merge type df to previously merged df and rename type
schl_type = school_data[["school_name","type"]]
merging_df2 = pd.merge(schl_type, schl_avg_rename, how="inner", on="school_name")
rename_schl = merging_df2.rename(columns={"type": "School Type"})

### School Budget

In [30]:
rename_schl["Budget per Student"]=""
rename_schl

# Calculate budget per student for each school
budget_student = rename_schl["Budget"]/rename_schl["Student Population"]

# Save budget per student in df and made a copy of df
rename_schl["Budget per Student"]=budget_student
df_copy = rename_schl.copy()
df_copy

,school_name,School Type,Student Population,Budget,Average Reading Score,Average Math Score,Budget per Student
0,Huang High School,District,2917,1910635,81.182722,76.629414,655.0
1,Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0
2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,600.0
3,Hernandez High School,District,4635,3022020,80.934412,77.289752,652.0
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0
5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,578.0
6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,582.0
7,Bailey High School,District,4976,3124928,81.033963,77.048432,628.0
8,Holden High School,Charter,427,248087,83.814988,83.803279,581.0
9,Pena High School,Charter,962,585858,84.044699,83.839917,609.0


### Passing Score (70+) Percentage

In [9]:
# Calculate the percentage of students with a passing reading score (70 or greater)

# Select passing reading scores and group scores by school by count of students
read_pass = school_data_complete.loc[school_data_complete["reading_score"]>=70]
read_students = read_pass.groupby("school_name").count()

# Calculate the student count as a percentage and make a dataframe
school_pass_read = read_students["reading_score"]*100/score_avg["size"]
schl_pass_read_df = pd.DataFrame(school_pass_read)

# Rename dataframe to reflect percentage of students who passed
schl_pass_read_rename = schl_pass_read_df.rename(columns={0:"Passed Reading(%)"})

In [10]:
# Calculate the percentage of students with a passing math score (70 or greater)

# Select passing math scores and group scores by school by count of students
math_pass = school_data_complete.loc[school_data_complete["math_score"]>=70]
students_math = math_pass.groupby("school_name").count()

# Calculate the student count as a percentage and make a dataframe
school_pass_math = students_math["math_score"]*100/score_avg["size"]
schl_pass_math_df = pd.DataFrame(school_pass_math)

# Rename dataframe to reflect percentage of students who passed
schl_pass_math_rename = schl_pass_math_df.rename(columns={0:"Passed Math(%)"})

In [37]:
# Calculate the percentage of students with overall passing score
rate_df = (school_pass_math+school_pass_read)/2
rate_df = pd.DataFrame(rate_df)
overall_rate = rate_df.rename(columns={0:"Overall Rate(%)"})

### School Summary DataFrame

In [38]:
# Merge previous dataframes to create a summary and rename school name rename_schl
summary_df = pd.merge(df_copy, schl_pass_read_rename, how="left", on=["school_name"])
summary_df2 = pd.merge(summary_df, schl_pass_math_rename, how="left", on=["school_name"])
summary_df3 = pd.merge(summary_df2, overall_rate, how="left", on=["school_name"])
school_rename = summary_df3.rename(columns={"school_name": "School Name"})

# Reorganize df
school_summary_df = school_rename[["School Name","School Type","Student Population","Budget","Budget per Student",
                              "Average Reading Score","Average Math Score","Passed Reading(%)","Passed Math(%)",
                               "Overall Rate(%)"]]
school_summary_df

,School Name,School Type,Student Population,Budget,Budget per Student,Average Reading Score,Average Math Score,Passed Reading(%),Passed Math(%),Overall Rate(%)
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,81.316421,65.683922,73.500171
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,80.739234,65.988471,73.363852
2,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,95.854628,93.867121,94.860875
3,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,80.862999,66.752967,73.807983
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,97.138965,93.392371,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,96.539641,93.867718,95.203679
6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,97.039828,94.133477,95.586652
7,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,81.933280,66.680064,74.306672
8,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,96.252927,92.505855,94.379391
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,95.945946,94.594595,95.270270


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [39]:
school_summary_df.sort_values("Overall Rate(%)", ascending=False).head()

,School Name,School Type,Student Population,Budget,Budget per Student,Average Reading Score,Average Math Score,Passed Reading(%),Passed Math(%),Overall Rate(%)
6,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,97.039828,94.133477,95.586652
14,Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,97.308869,93.272171,95.290520
9,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,95.945946,94.594595,95.270270
4,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,97.138965,93.392371,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,96.539641,93.867718,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [40]:
school_summary_df.sort_values("Overall Rate(%)").head()

,School Name,School Type,Student Population,Budget,Budget per Student,Average Reading Score,Average Math Score,Passed Reading(%),Passed Math(%),Overall Rate(%)
11,Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,80.220055,66.366592,73.293323
1,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,80.739234,65.988471,73.363852
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,81.316421,65.683922,73.500171
12,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,81.222432,66.057551,73.639992
13,Ford High School,District,2739,1763916,644.0,80.746258,77.102592,79.299014,68.309602,73.804308


## Math & Reading Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

### Grouping Grades

In [43]:
# student_id = school_data_complete.set_index("Student ID")
# student_id

# Filter scores by 9th grade
nine = school_data_complete.loc[school_data_complete["grade"]=="9th"]
nine_math_avg = nine.groupby("school_name").mean()
nine_math_avg_rename = nine_math_avg.rename(columns={"reading_score":"Grade 9 Reading Score",
                                                    "math_score":"Grade 9 Math Score"})

# Filter scores by 10th grade
ten = school_data_complete.loc[school_data_complete["grade"]=="10th"]
ten_math_avg = ten.groupby("school_name").mean()
ten_math_avg_rename = ten_math_avg.rename(columns={"reading_score":"Grade 10 Reading Score",
                            "math_score":"Grade 10 Math Score"})

# Filter scores by 11th grade
eleven = school_data_complete.loc[school_data_complete["grade"]=="11th"]
eleven_math_avg = eleven.groupby("school_name").mean()
eleven_math_avg_rename = eleven_math_avg.rename(columns={"reading_score":"Grade 11 Reading Score",
                                "math_score":"Grade 11 Math Score"})

# Filter scores by 12th grade
twelve = school_data_complete.loc[school_data_complete["grade"]=="12th"]
twelve_math_avg = twelve.groupby("school_name").mean()
twelve_math_avg_rename = twelve_math_avg.rename(columns={"reading_score":"Grade 12 Reading Score",
                                "math_score":"Grade 12 Math Score"})

### Merge Grades

In [ ]:
# Merging 9th & 10th grade
nine_ten_avg = pd.merge(nine_math_avg_rename, ten_math_avg_rename, how="left", on=["school_name"])

# Merging 11th & 12th grade
eleven_twelve_avg = pd.merge(eleven_math_avg_rename, twelve_math_avg_rename, how="left", on=["school_name"])

# Merging all grades
school_score_avg = pd.merge(nine_ten_avg, eleven_twelve_avg, how="left", on=["school_name"])

# Reset index and rename school name column
df_reset_index = school_score_avg.reset_index()
renaming_school = df_reset_index.rename(columns={"school_name":"School Name"})

### Average Math Score by Grade Summary 

In [45]:
# Filter grades to get math scores per school
math_avg = renaming_school[["School Name","Grade 9 Math Score","Grade 10 Math Score",
                             "Grade 11 Math Score","Grade 12 Math Score"]]
math_avg

,School Name,Grade 9 Math Score,Grade 10 Math Score,Grade 11 Math Score,Grade 12 Math Score
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


### Average Reading Score by Grade Summary 

In [46]:
# Filter grades to get reading scores per school
reading_avg = renaming_school[["School Name","Grade 9 Reading Score","Grade 10 Reading Score",
                             "Grade 11 Reading Score","Grade 12 Reading Score"]]
reading_avg

,School Name,Grade 9 Reading Score,Grade 10 Reading Score,Grade 11 Reading Score,Grade 12 Reading Score
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending, School Size,School Type

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

### Score & School Spending Summary

In [47]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [54]:
# Create a new data frame by locating the desired columns
scores_spending = school_summary_df[["Average Reading Score","Average Math Score",
                                     "Passed Reading(%)","Passed Math(%)","Overall Rate(%)"]]

# Add a new columns named Spending Ranges (Per Student) and binning based off budget per student
scores_spending["Spending Ranges (Per Student)"]= pd.cut(school_summary_df["Budget per Student"],
                                                         spending_bins,labels=group_names)
# Create a group based off of the bins
scores_spending = scores_spending.groupby("Spending Ranges (Per Student)").mean()
scores_spending.head()

C:\Users\tamal\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Average Reading Score,Average Math Score,Passed Reading(%),Passed Math(%),Overall Rate(%)
Spending Ranges (Per Student),,,,,
<$585,83.933814,83.455399,96.610877,93.460096,95.035486
$585-615,83.885211,83.599686,95.900287,94.230858,95.065572
$615-645,81.891436,79.079225,86.106569,75.668212,80.887391
$645-675,81.027843,76.997210,81.133951,66.164813,73.649382


### Score & School Size Summary

In [58]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_size = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [59]:
# Create a new data frame by locating the desired columns
size_score = school_summary_df[["Average Reading Score","Average Math Score",
                                "Passed Reading(%)","Passed Math(%)","Overall Rate(%)"]]

# Add a new columns named Spending Ranges (Per Student) and binning based off budget per student
size_score["School Size"]= pd.cut(school_summary_df["Student Population"], size_bins,labels=group_size)

# Create a group based off of the bins
size_score = size_score.groupby("School Size").mean()
size_score.head()

C:\Users\tamal\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Average Reading Score,Average Math Score,Passed Reading(%),Passed Math(%),Overall Rate(%)
School Size,,,,,
Small (<1000),83.929843,83.821598,96.099437,93.550225,94.824831
Medium (1000-2000),83.864438,83.374684,96.790680,93.599695,95.195187
Large (2000-5000),81.344493,77.746417,82.766634,69.963361,76.364998


### Score & School Type Summary

In [50]:
score_school = school_summary_df.copy()
score_school = score_school.groupby("School Type").mean()
score_school[["Average Reading Score", "Average Math Score", 
              "Passed Reading(%)", "Passed Math(%)", "Overall Rate(%)"]]

,Average Reading Score,Average Math Score,Passed Reading(%),Passed Math(%),Overall Rate(%)
School Type,,,,,
Charter,83.896421,83.473852,96.586489,93.620830,95.103660
District,80.966636,76.956733,80.799062,66.548453,73.673757
